In [8]:
import numpy as np
import os, sys, glob
import matplotlib.pyplot as plt
import open3d as o3d
from scipy.stats import chisquare
import pandas as pd

basedir = os.path.dirname(os.getcwd())
_py = os.path.join(basedir, 'py')
_data = os.path.join(basedir, 'data')
_images = os.path.join(basedir, 'images')

sys.path.insert(1, _py)
import lad
import figures

from dotenv import load_dotenv
load_dotenv()

%load_ext autoreload
%autoreload 2

%matplotlib qt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Ray tracing

Requisites:
* Raw files (blensor ouputs) shoul be placed in the root of the directory
* Raw files should be named `s1*.npy`, `s2*.npy`, etc
* Create a file with sensor positions in the root with name `scanner_pos.txt`. It should contain 4 columns:
    * sensor name i.e. `s1`, `s2`, etc
    * x coordinate
    * y coordinate
    * z coordinate

In [29]:
# mockname = 'single_combined_lite'
res = 0.01 # only to look in the correct file
debug = True
downsample_debug = 1
voxel_size = 0.20

aa = []
bins = np.linspace(0.09, 0.2, 10)

for i in bins:
    voxel_size = i
    m3att, meshfile, bia, voxk, lia, ws, resdir = lad.get_lad(mockname, res=res, debug=debug, downsample_debug=downsample_debug, voxel_size=voxel_size)
    tot = np.product(m3att.shape)
    fNi = np.sum(m3att == 1)/tot
    fNp = np.sum(m3att == 2)/tot
    aa.append([i, fNi, fNp])


/home/omar/myproj/planttech/data/single_57_combined_lite/mesh.ply
Stage 1: 1.483928536999997
9.45 9.629999999999999 6.029999999999999
Stage 2: 12.026305396999987
max --> [104, 106, 66]
min --> [0, 0, 0]
(105, 107, 67)
Stage 3: 3.428705949999994
Number of voxels ocupied by points cloud: 	 8951
Number of voxels ocupied by beam points cloud: 	 558523
Total number of voxels in plant regions: 	 752745
Number of voxels with attribute 1: 	 8951
Number of voxels with attribute 2: 	 550105
Number of voxels with attribute 3: 	 193689
Stage 4: 11.807528854999987
Stage 5: 0.5082777800000144
/home/omar/myproj/planttech/data/single_57_combined_lite/mesh.ply
Stage 1: 2.0697335290000183
9.404444444444444 9.71111111111111 6.0311111111111115
Stage 2: 10.192484158000013
max --> [91, 94, 58]
min --> [0, 0, 0]
(92, 95, 59)
Stage 3: 2.308150134000016
Number of voxels ocupied by points cloud: 	 8351
Number of voxels ocupied by beam points cloud: 	 413205
Total number of voxels in plant regions: 	 515660
Numb

In [30]:
aa = np.array(aa)
plt.plot(aa[:,0], aa[:,1], label='nI 57')
plt.plot(aa[:,0], aa[:,2], label='nP 57')
plt.plot(aa[:,0], aa[:,1]/(aa[:,1]+aa[:,2]), label='lai 57')
plt.legend()

In [24]:
#
mockname = 'single_57_combined_lite'
res = 0.01
debug = True
downsample_debug = 1
# scan_inc = 90

tracers = lad.get_all_traces(mockname, res=res, debug=debug, downsample_debug=downsample_debug)
# get_lad(mockname, res, debug=debug, downsample_debug=downsample_debug)

/home/omar/myproj/planttech/data/single_57_combined_lite/s1300000.npy
s1300000.npy 	 beam traces points in tree tree_0: 	 1500 --> 1203584
/home/omar/myproj/planttech/data/single_57_combined_lite/s1200000.npy
s1200000.npy 	 beam traces points in tree tree_0: 	 1500 --> 2637655
/home/omar/myproj/planttech/data/single_57_combined_lite/s1100000.npy
s1100000.npy 	 beam traces points in tree tree_0: 	 1500 --> 1847948
/home/omar/myproj/planttech/data/single_57_combined_lite/s0200000.npy
s0200000.npy 	 beam traces points in tree tree_0: 	 1500 --> 1019883
/home/omar/myproj/planttech/data/single_57_combined_lite/s0300000.npy
s0300000.npy 	 beam traces points in tree tree_0: 	 1500 --> 1252779
/home/omar/myproj/planttech/data/single_57_combined_lite/s0800000.npy
s0800000.npy 	 beam traces points in tree tree_0: 	 1500 --> 1406049
/home/omar/myproj/planttech/data/single_57_combined_lite/s0700000.npy
s0700000.npy 	 beam traces points in tree tree_0: 	 1500 --> 1563232
/home/omar/myproj/planttech

In [42]:
mockname = 'single_57_combined'
figures.show_beams(mockname, sample=50, tracers=False, res=3)

In [25]:
#
# mockname = 'single_57_combined_lite'
res = 0.01 # only to look in the correct file
debug = True
downsample_debug = 1
voxel_size = 0.13
# scan_inc = 130
# alpha = 0.2
# kbins = 2

m3att, meshfile, bia, voxk, lia, ws, resdir = lad.get_lad(mockname, res=res, debug=debug, downsample_debug=downsample_debug, voxel_size=voxel_size)

/home/omar/myproj/planttech/data/single_57_combined_lite/mesh.ply
Stage 1: 0.8710573379999857
9.49 9.75 5.98
Stage 2: 9.704872574000007
max --> [72, 74, 45]
min --> [0, 0, 0]
(73, 75, 46)
Stage 3: 0.994033839999986
Number of voxels ocupied by points cloud: 	 7088
Number of voxels ocupied by beam points cloud: 	 223753
Total number of voxels in plant regions: 	 251850
Number of voxels with attribute 1: 	 7088
Number of voxels with attribute 2: 	 216781
Number of voxels with attribute 3: 	 27981
Stage 4: 7.463481373999997
Stage 5: 0.5753146649999792


In [26]:
alphas_k = lad.alpha_k(bia, voxk, lia, ws, resdir, meshfile, show=False, klia=False, use_true_lia=True)

In [27]:

kbins = 2
# lads_min = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,2], 1)
# lads_max = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,4], 1)
lads_mid = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,6], 1)
lads_0 = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,6]*0+1, 1.0)
kmax = m3att.shape[2]
lads_mesh = lad.get_LADS_mesh(meshfile, voxel_size, kbins, kmax)

In [28]:
fig = plt.figure(figsize=(4, 6))
# plt.plot(lads_min[0:,1], lads_min[0:,0], marker='*', label='Estimated min')
# plt.plot(lads_max[0:,1], lads_max[0:,0], marker='*', label='Estimated max')
plt.plot(lads_mesh[0:,1], lads_mesh[0:,0], marker='s', c='k', label='Truth')
plt.plot(lads_mid[0:,1], lads_mid[0:,0], marker='*', label='Estimated mid')
plt.plot(lads_0[0:,1], lads_mid[0:,0], marker='*', label='Estimated 0')

plt.xlabel(r'LAD ($m^2/m^3$)')
plt.ylabel(r'Height ($m$)')
# plt.grid()
plt.legend()
# plt.show()
# plt.xlim(-2,2)

mockdir = os.path.join(_data, mockname)
resdir_name = '%s_%s' %('results', mockname)
resdir = os.path.join(mockdir, resdir_name)
savefig = os.path.join(resdir, 'LADs.png')
plt.savefig(savefig, dpi=200, bbox_inches='tight')

In [ ]:
plt.figure(figsize=(4, 6))
# plt.plot(lads_min[0:,1], lads_min[0:,0], marker='*', label='Estimated min')
# plt.plot(lads_max[0:,1], lads_max[0:,0], marker='*', label='Estimated max')
plt.plot(lads_mesh[0:,1], lads_mesh[0:,0], marker='s', c='r', label='Truth')
plt.plot(lads_mid[0:,1], lads_mid[0:,0], marker='*', label='Estimated mid')
plt.plot(lads_0[0:,1], lads_mid[0:,0], marker='*', label='Estimated 0')

plt.xlabel(r'LAD ($m^2/m^3$)')
plt.ylabel(r'Height ($m$)')
# plt.grid()
plt.legend()
# plt.show()
# plt.xlim(-2,2)

In [13]:
mockname = 'single'
downsample_debug = 20
voxel_size = 0.2
treename = 'tree_0'

mockdir = os.path.join(_data, mockname)
segtrees_dir_name = 'toy_trees'

tree = np.load('/home/omar/myproj/planttech/data/single/toy_trees/tree_0.npy')
tree = tree[::downsample_debug]
points = tree.T[5:8].T
beam_angles = tree.T[1:3].T # Pitch and Yaw

voxk = lad.get_voxk(points, voxel_size)

# Check if angles and weights are available
outdir_angs = os.path.join(mockdir, segtrees_dir_name, 'angles_%s_%s.npy' %(treename, str(1)))
outdir_ws = os.path.join(mockdir, segtrees_dir_name, 'weights_%s_%s.npy' %(treename, str(1)))

angs = np.load(outdir_angs)
angs = angs[::downsample_debug]
ws = np.load(outdir_ws)
ws = ws[::downsample_debug]
res = lad.alpha_k(beam_angles, voxk, angs, ws)

In [23]:
res[0:4][:,2]

array([-7.52392013,  2.5404234 , -0.10688612,  7.33053322])

In [ ]:
mockname = 'single'
mockdir = os.path.join(_data, mockname)
meshfile = os.path.join(mockdir, 'mesh.ply')

voxel_size = 0.4
kmax = 16
kbins = 3

lad.get_LADS_mesh(meshfile, voxel_size=voxel_size, kmax=kmax, kbins=kbins)

In [ ]:
np.full(2, 10)

In [ ]:
tr = np.load('/home/omar/myproj/planttech/data/single/results_single/tracers_tree_0_0.1.npy', allow_pickle=True)

In [ ]:
tr.shape